In [ ]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,017 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,544 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRe

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-03-20 04:07:12--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.29MB/s    in 0.2s    

2023-03-20 04:07:13 (6.29 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

### Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [ ]:
# Create the customers_table DataFrame
customers_df= df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customeer_id", "customer_count")
customers_df.show()

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   10270641|                 1|
|   18365872|                 1|
|   16711087|                 1|
|   10742726|                 2|
|   41169638|                 1|
|   43622307|                 1|
|   24540309|                 2|
|   28258386|                 1|
|   35329257|                 2|
|   14552054|                 1|
|   14529507|                 5|
|   45392827|                 5|
|   47282953|                 1|
|    8201930|                 1|
|   20109760|                 2|
|   16405801|                 4|
|   15056685|                21|
|   20840575|                 2|
|   39048303|                 1|
|    5596610|                 1|
+-----------+------------------+
only showing top 20 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0310824230|Advantage Flea Co...|
|0615553605|Pet Qwerks Treat ...|
|0684836483|250 Things You Ca...|
|0763004227|Golden Retriever ...|
|0764102885|Saint Bernards (B...|
|0764118269|  The Finch Handbook|
|0764144294|             Gerbils|
|0876051468|The Stray Cat Han...|
|0972585419|Feathered Phonics...|
|0975412817|The Notes Pocket ...|
|0977264815|ABC's of Behavior...|
|0982083513|Piece of Pie A La...|
|0982393555|Pill Wrap Paste -...|
|0983794804|Pathway Dog Journ...|
|0983794812|Pathway Cat Journ...|
|1083812550|Lees Wooden Air D...|
|1083813215|Lees Under Gravel...|
|1300347635|      Pet Herbal FPS|
|1300450991|Puppy Booster Her...|
|1300451041|Kitten Booster Ca...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+-----------+
|     review_id|customer_id|product_id|review_date|
+--------------+-----------+----------+-----------+
| REAKC26P07MDN|   28794885|B00Q0K9604| 2015-08-31|
|R3NU7OMZ4HQIEG|   11488901|B00MBW5O9W| 2015-08-31|
|R14QJW3XF8QO1P|   43214993|B0084OHUIO| 2015-08-31|
|R2HB7AX0394ZGY|   12835065|B001GS71K2| 2015-08-31|
| RGKMPDQGSAHR3|   26334022|B004ABH1LG| 2015-08-31|
|R1DJCVPQGCV66E|   22283621|B00AX0LFM4| 2015-08-31|
|R3V52EAWLPBFQG|   14469895|B00DQFZGZ0| 2015-08-31|
|R3DKO8J1J28QBI|   50896354|B00DIRF9US| 2015-08-31|
| R764DBXGRNECG|   18440567|B00JRCBFUG| 2015-08-31|
| RW1853GAT0Z9F|   50502362|B000L3XYZ4| 2015-08-31|
|R33GITXNUF1AD4|   33930128|B00BOEXWFG| 2015-08-31|
|R1H7AVM81TAYRV|   43534290|B001HBBQKY| 2015-08-31|
|R2ZOYAQZNNZZWV|   45555864|B007O1FHB0| 2015-08-31|
|R2FN1H3CGW6J8H|   11147406|B001P3NU30| 2015-08-31|
| RJB41Q575XNG4|    6495678|B00ZP6HS6S| 2015-08-31|
|R28W8BM1587CPF|    2019416|B00IP05CUA| 2015-08-31|
|R1II0M01NIG

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
from getpass import getpass
password = getpass('Enter database password')

Enter database password··········


In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dzagorac.c5y8do3arem7.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)